Import package (aku importir sejati)

In [111]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np
from datetime import timedelta

In [112]:
df = pd.read_excel("fmcg_sales_data_2023_2024_cleaned.xlsx")
df

,date,sku,product_name,category,base_price,promotion_flag,final_price,units_sold,stock_on_hand
0,2024-09-07,SKU004,Shampo 200ml,Perawatan Diri,25800,0,25800,53,68
1,2023-07-17,SKU033,Pasta Gigi 75g,Perawatan Diri,26000,0,26000,0,0
2,2023-03-17,SKU025,Teh Celup 25pcs,Minuman,34400,0,34400,43,51
3,2024-05-22,SKU014,Shampo 200ml,Perawatan Diri,6100,0,6100,7,8
4,2023-01-02,SKU004,Shampo 200ml,Perawatan Diri,25800,1,20110,75,81
...,...,...,...,...,...,...,...,...,...
50321,2023-06-07,SKU007,Sabun Mandi 100g,Perawatan Diri,34400,1,27317,26,32
50322,2024-06-15,SKU015,Teh Celup 25pcs,Minuman,26500,1,21356,9,25
50323,2024-08-16,SKU026,Keripik Kentang Balado 150g,Makanan Ringan,49500,1,37524,3,33
50324,2023-08-16,SKU006,Pasta Gigi 75g,Perawatan Diri,34200,1,25020,45,75


In [113]:
df['day_of_week'] = df['date'].dt.dayofweek
df['month'] = df['date'].dt.month


In [114]:
# Simpan hasil
result = []

In [115]:
# Loop per SKU
for sku in df['sku'].unique():
    temp = df[df['sku'] == sku].copy()
    temp = temp.sort_values('date')
    
    if len(temp) < 30:
        continue  # Data terlalu sedikit untuk evaluasi yang layak

    product_name = temp['product_name'].iloc[0]
    current_stock = temp['stock_on_hand'].iloc[-1]

    # Fitur dan target
    X = temp[['day_of_week', 'month', 'stock_on_hand']]
    y = temp['units_sold']

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

    # ---------- 1. Linear Regression ----------
    model_lr = LinearRegression()
    model_lr.fit(X_train, y_train)
    y_pred_lr = model_lr.predict(X_test)
    mae_lr = mean_absolute_error(y_test, y_pred_lr)
    r2_lr = r2_score(y_test, y_pred_lr)

    # ---------- 2. XGBoost ----------
    model_xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)
    model_xgb.fit(X_train, y_train)
    y_pred_xgb = model_xgb.predict(X_test)
    mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
    r2_xgb = r2_score(y_test, y_pred_xgb)

    # ---------- Forecast 7 Hari ke Depan ----------
    last_stock = temp['stock_on_hand'].iloc[-1]
    last_date = temp['date'].max()
    future_dates = [last_date + timedelta(days=i) for i in range(1, 8)]
    future_X = pd.DataFrame({
        'day_of_week': [d.dayofweek for d in future_dates],
        'month': [d.month for d in future_dates],
        'stock_on_hand': [last_stock] * len(future_dates)  # pakai base_price
    })

    # Pilih model dengan MAE lebih kecil
    if mae_lr <= mae_xgb:
        chosen_model = 'Linear Regression'
        forecast = model_lr.predict(future_X)
    else:
        chosen_model = 'XGBoost'
        forecast = model_xgb.predict(future_X)

    # Forecast final
    forecast = np.round(forecast).astype(int)
    forecast = np.clip(forecast, 0, None)

    avg_demand = forecast.mean()
    safety_stock = 1.5 * avg_demand * 3  # asumsi lead time 3 hari
    reorder_point = forecast[:3].sum() + safety_stock
    restock_needed = max(0, reorder_point - current_stock)

    # Simpan hasil
    result.append({
        'sku': sku,
        'product_name': product_name,
        'chosen_model': chosen_model,
        'avg_demand': round(avg_demand),
        'safety_stock': round(safety_stock),
        'reorder_point': round(reorder_point),
        'current_stock': round(current_stock),
        'restock_needed': round(restock_needed)
    })

In [116]:
# df_coba = pd.get_dummies(df, columns=['product_name', 'category'], drop_first=True)
# df_coba

In [117]:
# df_drop = df.drop(columns=['sku', 'product_name','category'])  # atau nama kolom lain yg relevan
# df_drop_coba = df_coba.drop(columns=['sku'])  # atau nama kolom lain yg relevan
# df_drop.corr()['units_sold'].sort_values()
# df_drop_coba.corr()['units_sold'].sort_values()

In [118]:
# 'mae_lr': round(mae_lr, 2),
# 'r2_lr': round(r2_lr, 2),
# 'mae_xgb': round(mae_xgb, 2),

 # 'r2_xgb': round(r2_xgb, 2),

In [119]:
print("Linear Regression Performance:\n","MAE:", mae_lr,"\n R2 Score:", r2_lr)
print("================================")
print("XGBoost Performance:\n","MAE:", mae_xgb,"\n R2 Score:", r2_xgb)

Linear Regression Performance:
 MAE: 6.847312245809177 
 R2 Score: 0.8515099022837913
XGBoost Performance:
 MAE: 7.292909622192383 
 R2 Score: 0.8227438926696777


In [120]:
forecast_df = pd.DataFrame(result)
forecast_df.head()

,sku,product_name,chosen_model,avg_demand,safety_stock,reorder_point,current_stock,restock_needed
0,SKU004,Shampo 200ml,Linear Regression,27,121,201,37,164
1,SKU033,Pasta Gigi 75g,XGBoost,10,44,75,22,53
2,SKU025,Teh Celup 25pcs,XGBoost,2,10,15,6,9
3,SKU014,Shampo 200ml,Linear Regression,11,51,84,23,61
4,SKU046,Susu UHT 1L,XGBoost,3,15,24,14,10
